In [1]:
import pandas as pd

## 과제 1: Datetime Basics

In [2]:
transactions = pd.read_csv('./data/retail/transactions.csv', parse_dates=['date'])
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [3]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [4]:
transactions['date'].max()

Timestamp('2017-08-15 00:00:00')

In [5]:
# Difference between date and max-date
transactions['time_to_last_date'] = transactions['date'].max() - transactions['date']

# Dateparts
transactions['year'] = transactions['date'].dt.year
transactions['month'] = transactions['date'].dt.month
# 월요일(0), 일요일(6)
transactions['day_of_week'] = transactions['date'].dt.weekday

# Format Date
transactions['date'] = transactions['date'].dt.strftime('%Y-%B-%d')

transactions.head()

,date,store_nbr,transactions,time_to_last_date,year,month,day_of_week
0,2013-January-01,25,770,1687 days,2013,1,1
1,2013-January-02,1,2111,1686 days,2013,1,2
2,2013-January-02,2,2358,1686 days,2013,1,2
3,2013-January-02,3,3487,1686 days,2013,1,2
4,2013-January-02,4,1922,1686 days,2013,1,2
